## Install Packages

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow import Tensor
print("Tensorflow version is ", tf.__version__)
print('Keras version      : ',keras.__version__)
import numpy as np
import os, sys
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import h5py as h5
from sklearn.metrics import classification_report, confusion_matrix
import random
import time
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from tensorflow.keras.layers import  Dropout, Activation, GlobalAveragePooling1D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, Adadelta
from tensorflow.keras.layers import Reshape, Dense, Flatten, Add
from tensorflow.keras.activations import relu
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, History
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from random import shuffle
import glob
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle
from progressbar import ProgressBar
import matplotlib.pyplot as plt

## Download the 2018.01 Dataset from Deepsig
Deepsig has released multiple data sets that can be found here https://www.deepsig.io/datasets

This data set contains RF data with 24 different modulations at various SNR. Each of the 2555904 data inputs is 1024 samples long of complex (I Q) data.


In [ ]:
 !wget http://opendata.deepsig.io/datasets/2018.01/2018.01.OSC.0001_1024x2M.h5.tar.gz
 !pwd
 !ls
 !tar -xvzf 2018.01.OSC.0001_1024x2M.h5.tar.gz
 !ls
 !rm 2018.01.OSC.0001_1024x2M.h5.tar.gz 

## 2018 Dataset 

### Read in RF Data
3 Arrays will be created. <br>
myData holds the 1024 I and Q time values for each input sample. <br>
myMods holds the one hot encoded RF class for each sample.<br>
mySNRs holds the SNR value for each sample.<br>



In [ ]:
#Note this is needed to aviod a tensorFlow memory issue
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

data_file = '/workspace/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5'
file_handle = h5.File(data_file,'r+')

myData = file_handle['X'][:]  #1024x2 samples 
myMods = file_handle['Y'][:]  #mods 
mySNRs = file_handle['Z'][:]  #snrs  

print(np.shape(myData))
print(np.shape(myMods))
print(np.shape(mySNRs))
file_handle.close()

np.random.seed(0)

### List the SNRs


In [ ]:
snrs = list(np.unique(mySNRs.T[0]))  
print(snrs)

### Define the Modulaton classes

In [ ]:
mods = [
    'OOK',      '4ASK',      '8ASK',      'BPSK',   'QPSK',    '8PSK',
    '16PSK',    '32PSK',     '16APSK',    '32APSK', '64APSK',  '128APSK',
    '16QAM',    '32QAM',     '64QAM',     '128QAM', '256QAM',  
    'AM-SSB-WC','AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK','OQPSK']

num_classes = np.shape(mods)[0]
print("The number of classes is ", num_classes)


### Examine RF input samples
The samples in data set are ordered by class, let's print out one example from each class.

In [ ]:
#turn off warning about more than 10 figures plotted
plt.rcParams.update({'figure.max_open_warning': 0})

def my_range(start, end, step):
    while start <= end:
        yield start
        start += step

size = np.size(myData, axis = 0)
step = size//24

for x in my_range(100000, (size-1), step):
  plt.figure()
  plt.suptitle( mods[np.argmax(myMods[x])])
  plt.plot(myData[x,:,0])
  plt.plot(myData[x,:,1])

### Examine Input Data Range
The input datat is close to being centered around 0, and since the standard deviation is around 1.57, most of the data lies between -6 an +6 with some outliers. 

In [ ]:
print ("Max value of the data set = ", np.max(myData))
print ("Min value of the data set = ", np.min(myData))
print ("Mean value of the data set = ", np.mean(myData))
print ("Standard Deviation of the data set ", np.std(myData) )

Lets examine where the outlyers are comming from. We will see below that all data is between -5 an 5 expect for AM-SSB-WC and the AM-SSB-SC modulations

In [ ]:
len = (myData.shape[0])
limit = 5
Max = np.zeros(len)
Min = np.zeros(len)
for i in range(0,len):
  Max[i] = (np.max(myData[i,:,0]))
  Min[i] = (np.min(myData[i,:,0]))
  if(Max[i] > limit or Min[i] < -limit):
   print ("index =", i, mods[np.argmax(myMods[i])])
plt.figure()
plt.plot(Max)
plt.plot(Min)

### Remove AM-SSB-WC and AM-SSB_SC from the data set
If we leave the AM-SSB-WC and AM-SSB-SC modulations in the data set we will see lower accuracy after quantizing the model to INT8. This is becuase we can more accurately quantize the floating point input data if it is over a smaller range with fewer outlyers as in seen in the other modulations 

In the next step we will remove these two modulations from the data set. If you want to leave these modulations in, you can skip the next step. Leaving these in will cause an additonal 5% accuracy drop after quantizing.

In [ ]:
#Skip this entire panel if you want to leave AM-SSB-WC and AM-SSB-SC modulations in the data set
myData = np.concatenate((myData[0:1810432], myData[2023424:2555904]),axis=0)
mySNRs = np.concatenate((mySNRs[0:1810432], mySNRs[2023424:2555904]),axis=0)
myMods = np.concatenate((myMods[0:1810432], myMods[2023424:2555904]),axis=0)

#re-onehot encode myMods to 22 from 24
length = (np.size(myMods, axis=0))
temp = np.concatenate((myMods[:,0:17],myMods[:,19:24]), axis=1)
myMods = temp

mods = [
    'OOK',      '4ASK',      '8ASK',      'BPSK',   'QPSK',    '8PSK',
    '16PSK',    '32PSK',     '16APSK',    '32APSK', '64APSK',  '128APSK',
    '16QAM',    '32QAM',     '64QAM',     '128QAM', '256QAM',  
    'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK','OQPSK']

num_classes = np.shape(mods)[0]
print("The number of classes is ", num_classes)


print(np.shape(myData))
print(np.shape(mySNRs))
print(np.shape(myMods))

print ("Max value of the data set = ", np.max(myData))
print ("Min value of the data set = ", np.min(myData))
print ("Mean value of the data set = ", np.mean(myData))
print ("Standard Deviation of the data set ", np.std(myData) )


### Now lets look at how the SNRs are distributed across the data set
As you can see each SNR appears an equal number of times across the data set


In [ ]:
plt.figure()
plt.suptitle("SNR Distribution")
plt.hist(mySNRs, bins = [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32]) 
plt.show()

### Reshape RF data to 2D Matrix
We will reshape both the I and Q data from a 1024 long vector to 2D 1024x1 matrix to be conpatabile with 2D convolution commands supported by the DPU

In [ ]:
myData = myData.reshape(myData.shape[0], 1024, 1, 2) 

### Slpit Data into Trainnig and Validation set
We will use 80% of the data for the Training set and 20% for the Test set. 
The random_state input to the the train_test_split function is set to 0, which means the 80/20 split will be done in a repeatable manner. 

In [ ]:
X_train ,X_test ,Y_train ,Y_test, Z_train, Z_test =train_test_split(myData, myMods, mySNRs, test_size=0.2, random_state=0)
print (np.shape(X_test))
print (np.shape(Y_test))
print (np.shape(Z_test))
print (np.shape(X_train))
print (np.shape(Y_train))
print (np.shape(Z_train))


## Build a Simple Resnet  Model 


This resnet model was inspired by the model disussed in the following paper <br>
Over-the-Air Deep Learning Based Radio Signal Classification, <br>
IEEE JOURNAL OF SELECTED TOPICS IN SIGNAL PROCESSING, VOL. 12, NO. 1, <br>
FEBRUARY 2018 

In [ ]:
input_shp = list(X_train.shape[1:])
print("Dataset Shape={0} CNN Model Input layer={1}".format(X_train.shape, input_shp))
classes = mods

Here we construct a resnet block which will be used multiple times in our model. In order to be compatabile with the Vitis-AI compilation tools we need to use a square kernel size (2x2) for the MaxPooling layer. Since the data is rectangular 1024x1, 512x1, ... we have enabled padding so the a square kernel size can be used.

In [ ]:
def resnet_block(input_data, filters, conv_size):
  x = Conv2D(filters, 1, activation='relu', padding='same')(input_data)
  x = Conv2D(filters, conv_size, activation='relu', padding='same')(x)
  x = Conv2D(filters, conv_size, activation=None, padding='same')(x)
  x = Add()([x, input_data])
  x = Activation('relu')(x)
    
  y = Conv2D(filters, conv_size, activation='relu', padding='same')(x)
  y = Conv2D(filters, conv_size, activation=None, padding='same')(y)
  y = Add()([y, x])
  y = Activation('relu')(y)
  
  z = MaxPooling2D(2, strides = (2,1), padding = 'same') (y)
  return z

Here we construct a model with 6 resnet_blocks.
Note the 4x4 reshape layer before the GlobalAveragePooling2D layer. This is needed to reshape the 16x1 data shape to 4x4 to be compatabile with the Vitis-AI compilation tools, as the Xilinx DPU only supports square data shapes for this layer. 

In [ ]:
num_resnet_blocks = 6
num_filters = 32
rf_input = Input(shape=input_shp, name = 'rf_input')

x = Conv2D(num_filters, (5,1), activation='relu', padding='same')(rf_input)

for i in range(num_resnet_blocks):
    x = resnet_block(x, num_filters, (5,1))

x = Conv2D(num_filters, (5,1), activation='relu', padding = 'same')(x)
x = Reshape((4,4,num_filters), input_shape = (16,1,num_filters)) (x)
x = GlobalAveragePooling2D()(x)
dense_1 = Dense(256, activation='relu')(x)
dropout_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(128, activation='relu')(dropout_1)
dropout_2 = Dropout(0.5)(dense_2)
dense_3 = Dense(num_classes)(dropout_2)          
softmax = Activation('softmax', name = 'softmax')(dense_3)


model = keras.Model(rf_input, softmax)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

print(model.summary())

Lets verify that our model is working. Because the weight are unitialize, all the probabilites will be close to a random guess of 0.045 (1/22)

In [ ]:
predict = model.predict(X_test[0:1])
print(predict)

## TRAIN

### Set up batch and epoch size.

In [ ]:
nb_epoch = 100     # number of epochs to train on
batch_size = 1024  # training batch size

### Early stoping callback 
Will end training after 5 epochs with no accuracy improvement on test data set

In [ ]:
### Callback
checkpoint_dir = 'resnet_checkpoints'
os.mkdir(checkpoint_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath =checkpoint_dir + '/weights.{epoch:03d}.h5', 
                                                 verbose = 1,
                                                 save_best_only=True, 
                                                 save_weights_only=False,
                                                 mode='auto')
earlystopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        patience = 5,
        mode='auto',
        verbose = 1)

### Start Training

In [ ]:
history = model.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=1,
    validation_data=(X_test, Y_test),
    callbacks = [ cp_callback,earlystopping_callback]
    )


## Training Completed
Reload the best weights once training is fininsed. 
Your best checkpoint may be different than 032.h5, due to the fact that the intial weight values are randomized.

In [ ]:
best_checkpoint = checkpoint_dir + '/weights.032.h5'
model.load_weights(best_checkpoint)
!mkdir -p fp_model
model.save ('fp_model/resnet_fp_model.h5')

In [ ]:
model = tf.keras.models.load_model('fp_model/resnet_fp_model.h5')

## Evaluate Model Performance

In [ ]:
# Show simple version of performance
score = model.evaluate(X_test, Y_test,  verbose=0, batch_size=batch_size)
print(score)

The Top1 accuracy should be close to 62%


### Confusion Matrix


In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.figure(figsize = (15,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    label_len = np.shape(labels)[0]
    tick_marks = np.arange(label_len)
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Plot confusion matrix
test_Y_hat = model.predict(X_test, batch_size=batch_size)
conf = np.zeros([num_classes,num_classes])
confnorm = np.zeros([num_classes,num_classes])
for i in range(0,X_test.shape[0]):
    j = list(Y_test[i,:]).index(1)
    k = int(np.argmax(test_Y_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,num_classes):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

In [ ]:
plot_confusion_matrix(confnorm, labels=classes)

### Generate Classification Report

In [ ]:
Y_pred = model.predict(X_test,batch_size=batch_size)
y_pred = np.argmax(Y_pred, axis = 1)
y_actual = np.argmax(Y_test, axis = 1)
classificationreport_fp = classification_report(y_actual,y_pred, target_names=mods)
print(classificationreport_fp)

Precision Measures the  Accuracy of the positive predictions.
Precision = TP/(TP + FP)

Recall Measures the fraction of positives that were correctly identified.
Recall = TP/(TP+FN)

The F1 score is measure of a weighted harmonic mean of precision and recall. 

Support is the number of values for each class.

Looking at the f1- scores for the different classes, we can see that the model is more accuracte with some classses than others.
For example, the model is not able to correctly inentify the hiher order PSK and QAM modulations as other classes.   

### Accuracy vs. SNR
Now lets see how the model accuracy is effcted by SNR.

In [ ]:
batchsize = 1024
progress = ProgressBar()
snrlist = np.unique(Z_test)
acc_snr_arr = []

# interate over SNRs
for snr in progress(snrlist):
    acc_arr = []
    i_SNR = np.where(Z_test==snr)
    X_SNR = X_test[i_SNR[0],:,:]
    Y_SNR = Y_test[i_SNR[0],:]
    X_SNR_len = np.shape(X_SNR)[0]
    total_batches = int(X_SNR_len/batchsize)
    
    for i in (range(0, total_batches)):
        x_batch, y_batch = X_SNR[i*batchsize:i*batchsize+batchsize], Y_SNR[i*batchsize:i*batchsize+batchsize]
        
        # model prediction
        pred = model.predict(x_batch)
        
        #Pediction values are onehote, corresponding to indices representing different modulation types
        pred_ind = np.argmax(pred, axis=1)
        expected_ind = np.argmax(y_batch, axis=1)
        matches  = sum(np.equal(pred_ind, expected_ind))
        acc      = matches/batchsize
        acc_arr.append(acc)

    # Average the per-batch accuracy values
    accuracy = np.mean(acc_arr)
    acc_snr_arr.append(accuracy)
    print("SNR: ", snr, "accuracy", accuracy)

In [ ]:
plt.figure(figsize=(1,1))
plt.show()
fig= plt.figure(figsize=(10,8))
plt.plot(snrlist, acc_snr_arr, 'bo-', label='accuracy')
plt.ylabel('Accuracy')
plt.xlabel('SNR')
plt.title("Accuracy vs, SNR for Floating Point Model")
plt.legend()
plt.axis([-22, 32, 0, 1.0])
plt.grid()

Are you can see accurccy SNRs below -10db the Top1 accuracy is no better than a random guess (1/24), and once SNR is above 10db the Top1 accuracy is over 98%.

## Vitis AI
The Vitis-AI tools will be used the Quantize and Compile the model for accleration on the DPU. <br>
Vitis-AI 1.3 now natively supports keras in TensorFlow2, and we can directly read in the .h5 model. 

## Quantize Model to INT8
The Vitis-AI Quantizer uses a  small set of unlabeled samples to analyze the distribution of the activations. We will use 1000 input samples from the test set. <br>

The quantized.h5 model that is produced will be used as input to the Vitis-AI Quantizer

In [ ]:
# reload the model in case it was closed
!ls -l fp_model/
model = tf.keras.models.load_model('fp_model/resnet_fp_model.h5')
 
from tensorflow_model_optimization.quantization.keras import vitis_quantize
quantizer = vitis_quantize.VitisQuantizer(model)
quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:1000])

# Save the model
!mkdir -p quantize_results
quantized_model.save('quantize_results/quantized_model.h5')
!ls -l quantize_results

### Load and Compile Model for Evaluation
We can now load and recompile the INT8 model and run evaluations to compare with the floating point model

### Evalute  Model INT8 Performance

In [ ]:
# load quantized model
from tensorflow_model_optimization.quantization.keras import vitis_quantize
with vitis_quantize.quantize_scope():
  q_model = tf.keras.models.load_model('quantize_results/quantized_model.h5')

q_model.compile(loss='categorical_crossentropy', metrics=["accuracy"])

score = q_model.evaluate(X_test, Y_test,  verbose=0, batch_size=1024)
print(score)

The Overall Top-1 score has gone down by about 5% due to quantization

### Classification Report for INT8 Model

In [ ]:
batch_size = 128
Y_pred = q_model.predict(X_test,batch_size=batch_size)
y_pred = np.argmax(Y_pred, axis = 1)
y_actual = np.argmax(Y_test, axis = 1)
classificationreport_int8 = classification_report(y_actual,y_pred, target_names=mods)
print(classificationreport_int8)

### Accuracy vs SNR for INT8 Model

In [ ]:
batchsize = 128
progress = ProgressBar()
snrlist = np.unique(Z_test)
acc_snr_arr = []

# interate over SNRs
for snr in progress(snrlist):
    acc_arr = []
    i_SNR = np.where(Z_test==snr)
    X_SNR = X_test[i_SNR[0],:,:]
    Y_SNR = Y_test[i_SNR[0],:]
    X_SNR_len = np.shape(X_SNR)[0]
    total_batches = int(X_SNR_len/batchsize)
    
    for i in (range(0, total_batches)):
        x_batch, y_batch = X_SNR[i*batchsize:i*batchsize+batchsize], Y_SNR[i*batchsize:i*batchsize+batchsize]
        
        # model prediction
        pred = q_model.predict(x_batch)
        
        #Pediction values are 0-24, corresponding to indices representing different modulation types
        pred_ind = np.argmax(pred, axis=1)
        expected_ind = np.argmax(y_batch, axis=1)
        matches  = sum(np.equal(pred_ind, expected_ind))
        acc      = matches/batchsize
        acc_arr.append(acc)

    # Average the per-batch accuracy values
    accuracy = np.mean(acc_arr)
    acc_snr_arr.append(accuracy)
    print("SNR: ", snr, "accuracy", accuracy)

In [ ]:
plt.figure(figsize=(1,1))
plt.show()
fig= plt.figure(figsize=(10,8))
plt.plot(snrlist, acc_snr_arr, 'bo-', label='accuracy')
plt.ylabel('Accuracy')
plt.xlabel('SNR')
plt.title("Accuracy vs, SNR for INT8 Model")
plt.legend()
plt.axis([-22, 32, 0, 1.0])
plt.grid()

The Accuracy vs SNR looks very similar to the floating point model, expect the accuracy is down by about 5% for higher SNRs from the floating point model

## Optional Step Quantizer Fine Tuning
The Vitis-AI quantizer supports fine tuning, which can be used to recover some of the accuracy lost when quantizing. 
Fine tuning does use the entire training data set, and this step will take a long time to run. If you want to skip this step you can move on to compiling the model for the DPU.


Quantize finetuning is similar to float model finetuning. The difference is that quantize finetuning uses the APIs of the vai_q_tensorflow2 to rewrite the float graph to a quantized model before the training starts. 

When quantize fine tuning it is often beneifical to use a smaller learning rate. For the orginal floating point training we used the default learning rate of the Adam optimzer which is 0.001. For Fine tuning we will use a learning rate of 0.0001.

In [ ]:
# load the orginal floating point model
model = tf.keras.models.load_model('fp_model/resnet_fp_model.h5')
 
# *Call the vai_q_tensorflow2 api to create the quantize training model
from tensorflow_model_optimization.quantization.keras import vitis_quantize
quantizer = vitis_quantize.VitisQuantizer(model)
model = quantizer.get_qat_model()

model.compile(
#use new learning rate
optimizer= Adam(learning_rate=0.0001),
loss= 'categorical_crossentropy',
metrics=["accuracy"])

print(model.summary())
    
# Start the training/finetuning
history = model.fit(X_train,Y_train,                
  callbacks = [
  keras.callbacks.ModelCheckpoint(
  filepath='./quantize_train/',
  save_best_only=True,
  monitor='categorical_crossentropy',
  mode = 'auto',
  verbose=1)])

# Save the model
model.save('quantize_results/fine_tuned_model.h5')


### Evalute Model INT8 Performance after Fine Tuning

In [ ]:
# load quantized model
from tensorflow_model_optimization.quantization.keras import vitis_quantize
with vitis_quantize.quantize_scope():
  ft_model = tf.keras.models.load_model('quantize_results/quantized_model.h5')

ft_model.compile(loss='categorical_crossentropy', metrics=["accuracy"])

score = ft_model.evaluate(X_test, Y_test,  verbose=0, batch_size=1024)
print(score)

After fine tuning the accuarcy improved to, you should that the  accuarcy has improved to around 58.8%.

### Accuracy vs SNR for INT8 Model after Fine Tuning

In [ ]:
batchsize = 128
progress = ProgressBar()
snrlist = np.unique(Z_test)
acc_snr_arr = []

# interate over SNRs
for snr in progress(snrlist):
    acc_arr = []
    i_SNR = np.where(Z_test==snr)
    X_SNR = X_test[i_SNR[0],:,:]
    Y_SNR = Y_test[i_SNR[0],:]
    X_SNR_len = np.shape(X_SNR)[0]
    total_batches = int(X_SNR_len/batchsize)
    
    for i in (range(0, total_batches)):
        x_batch, y_batch = X_SNR[i*batchsize:i*batchsize+batchsize], Y_SNR[i*batchsize:i*batchsize+batchsize]
        
        # model prediction
        pred = ft_model.predict(x_batch)
        
        #Pediction values are 0-24, corresponding to indices representing different modulation types
        pred_ind = np.argmax(pred, axis=1)
        expected_ind = np.argmax(y_batch, axis=1)
        matches  = sum(np.equal(pred_ind, expected_ind))
        acc      = matches/batchsize
        acc_arr.append(acc)

    # Average the per-batch accuracy values
    accuracy = np.mean(acc_arr)
    acc_snr_arr.append(accuracy)
    print("SNR: ", snr, "accuracy", accuracy)

## Compile Model for DPU
The Vitis-AI compiler reads in the quantized model and generates an xmodel file which the instruction set for the Xilinx Deep Learning Processor (DPU). The arhictecture option (-a) is used to specify a json file which indicates which hw target the DPU is being compiled for.

In [ ]:
# Select Quantized or Quantize FineTuned model
model = quantize_results/fine_tuned_model.h5
#model = quantize_results/quantized_model.h5

# Select HW Target
#For ZCU104
!vai_c_tensorflow2 -m quantize_results/quantized_model.h5 -a /opt/vitis_ai/compiler/arch/DPUCZDX8G/ZCU104/arch.json -o vai_c_output -n rfClassification --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"

#For ZCU102 
#!vai_c_tensorflow2 -m quantize_results/quantized_model.h5 -a /opt/vitis_ai/compiler/arch/DPUCZDX8G/ZCU102/arch.json -o vai_c_output -n rfClassification --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"

#For Alveo U50
#!vai_c_tensorflow2 -m quantize_results/quantized_model.h5 -a /opt/vitis_ai/compiler/arch/DPUCAHX8H/U50/arch.json -o vai_c_output -n rfClassification --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"

#For Versal VCK190
#!vai_c_tensorflow2 -m quantize_results/quantized_model.h5 -a /opt/vitis_ai/compiler/arch/DPUCVDX8G/VCK190/arch.json -o vai_c_output -n rfClassification --options "{'cpu_arch':'arm64', 'mode':'normal', 'save_kernel':''}"

## Generate Graph Visualization with xir tool.
You will see a compiler message about the number of  subgraphs:
Total device subgraph number 3, DPU subgraph number 1 <br>
This means that are 3 subgraphs created, 1 for the input layer, 1 for for everything up the softmax layer (which runs on the DPU), and one for the softmax. <br>

The softmax layer can  optionally be acclerated in programmable logic, however in this tutorial we will implement the softmax layer on the CPU.

You can use the the xir command generate a .png file to visulize the graph layers.

In [ ]:
!xir png /workspace/vai_c_output/rfClassification.xmodel xmodel.png

### Write out  samples  of Test Data to be used later for HW testing
The python function we will run in the target board will read in these numpy files containing the RF data, class, and SNR.

In [ ]:
np.save('/workspace/rf_input.npy', X_test[0:1000,:,:])
np.save('/workspace/rf_classes.npy', Y_test[0:1000])
np.save('/workspace/rf_snrs.npy', Z_test[0:1000])

Now that a dpu xmodel file has been created you are ready to run on target board. You will need to copy the above 3 files, and the xmodel file from the compiler to your target board.

You can close this notebook by entering CtrlC at the console, close the docker container by entering CtrlD, and the proceed with the Tutorial readme instructions.